In [4]:
import numpy as np
import pandas as pd
import math
from scipy.stats import *
import scipy as sp
import matplotlib.pyplot as plt

# 그래프가 Jupyter notebook 위에 표시
%matplotlib inline
# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3

# Dataframe의 출력을 소수점 이하 3자리로 제한
pd.set_option('precision', 3)

'%.3f'

In [2]:
# 여러 변수 출력 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
df = pd.read_csv('./data/ch2_scores_em.csv',
                index_col='student number')
en_scores = np.array(df['english'])[:10]
ma_scores = np.array(df['mathematics'])[:10]
scores_df = pd.DataFrame({'english':en_scores,
                         'mathematics':ma_scores},
                         index=pd.Index(['A','B','C','D','E','F','G','H','I','J'],
                                       name='student'))
scores_df

,english,mathematics
student,,
A,42,65
B,69,80
C,56,63
D,41,63
E,57,76
F,48,60
G,65,81
H,49,66
I,65,78


# CHAPTER 03 2차원 데이터 정리
- 3.1 두 데이터 사이의 관계를 나타내는 지표
- 3.2 2차원 데이터의 시각화
- 3.3 앤스컴의 예

## 두 데이터 사이의 관계를 나타내는 지표

### 공분산
- 두 변수 사이의 관계성 척도
- 예) 영어/수학 점수의 공분산표 
    - 직사각형 가로 : 영어 점수의 편차
    - 직사각형 세로 : 수학 점수의 편차
    
<img src='./images/3_1_0001.png' width='500px' align='left'>

In [15]:
## 공분산 구하기
# 방법 1. 편차 직접 계산하여 공분산 구하기

print('1. 편차 직접 계산하여 공분산 구하기')
summary_df = df.copy()
summary_df['eng_dev.'] = summary_df['english'] - summary_df['english'].mean() # 영어점수 편차
summary_df['math_dev.'] = summary_df['mathematics'] - summary_df['mathematics'].mean() # 수학점수 편차
summary_df['productOfDev.'] = summary_df['eng_dev.'] * summary_df['math_dev.'] # 각 공분산 구하기
print('영어점수, 수학점수, 영어점수편차, 수학점수편차, 각 공분산 테이블')
summary_df.head()

print('공분산',summary_df['productOfDev.'].mean())
 # 공분산
# 결과 : 58.486(>0)이므로 양의 상관관계
print('----------------------------------------------')

# 방법 2. numpy의 공분산 함수 : cov(data1, data2, ddof=)
# 반환값 : 공분산 행렬 
# 반환 행렬 중 [0,1]과 [1,0] 의 원소가 공분산 값
print('2. numpy.cov()으로 공분산 구하기')
cov_mat = np.cov(df['english'],df['mathematics'], ddof=0) # 모집단의 공분산
cov_mat
print('대각요소(공분산) :',cov_mat[0,1],cov_mat[1,0])
print('영어과목의 분산 :',cov_mat[0,0])
print('수학과목의 분산 :',cov_mat[1,1])
print('영어과목의 분산 확인 :',np.var(df['english'],ddof=0))
print('수학과목의 분산 확인 :',np.var(df['mathematics'],ddof=0))

1. 편차 직접 계산하여 공분산 구하기
영어점수, 수학점수, 영어점수편차, 수학점수편차, 각 공분산 테이블


,english,mathematics,eng_dev.,math_dev.,productOfDev.
student number,,,,,
1,42,65,-16.38,-13.88,227.354
2,69,80,10.62,1.12,11.894
3,56,63,-2.38,-15.88,37.794
4,41,63,-17.38,-15.88,275.994
5,57,76,-1.38,-2.88,3.974


공분산 58.4856
----------------------------------------------
2. numpy.cov()으로 공분산 구하기


array([[94.116, 58.486],
       [58.486, 69.386]])

대각요소(공분산) : 58.485600000000005 58.485600000000005
영어과목의 분산 : 94.1156
수학과목의 분산 : 69.38560000000001
영어과목의 분산 확인 : 94.11560000000001
수학과목의 분산 확인 : 69.38559999999995


### 상관계수
- 공분산 : 각 데이터의 단위 제곱이므로 직관적으로 비교,이해가 어려움
    - 예시) 키와 점수간의 공분산 (cm X 점수)
- 상관계수 : 단위에 의존하지 않는 상관성을 나타내는 지표
    - 공분산𝘅𝘆 ÷ (표준편차𝘅)(표준편차𝘆) -> 단위에 의존하지 않음
    - 양의 상관관계 : 1에 가까움
    - 음의 상관관계 : -1에 가까움
    - 무상관관계 : 0

<img src='./images/3_1_0002.png' width='500px' align='left'>  

In [34]:
## 상관계수 구하기
# 방법 1. 공분산𝘅𝘆 ÷ (표준편차𝘅)(표준편차𝘆)으로 계산하기
print('1. np.cov(x,y,ddof=0)-----------------')
np_cov_arr = np.cov(df['english'],df['mathematics'],ddof=0)/ (np.std(df['english']) * np.std(df['mathematics']))
np_cov_arr
print(f'- df[\'english\'] : {np_cov_arr[0,0]}')
print(f'- df[\'mathematics\'] : {np_cov_arr[1,1]}')
print(f'- 상관계수 : {np_cov_arr[0,1]}, {np_cov_arr[1,0]}')

print()
print('2. np.corrcoef(x,y)-----------------')
# 방법 2. np.corrcoef(x,y) 
np_corr_arr = np.corrcoef(df['english'],df['mathematics'])
np_corr_arr
print(f'- df[\'english\'] : {np_corr_arr[0,0]}')
print(f'- df[\'mathematics\'] : {np_corr_arr[1,1]}')
print(f'- 상관계수 : {np_corr_arr[0,1]}, {np_corr_arr[1,0]}')

print()
print('3. df.corr()-----------------')
# 방법 3. pandas.DataFrame.corr()
df_corr_DF = df.corr()
df_corr_DF

1. np.cov(x,y,ddof=0)-----------------


array([[1.165, 0.724],
       [0.724, 0.859]])

- df['english'] : 1.1646518840307354
- df['mathematics'] : 0.8586256663571502
- 상관계수 : 0.7237414863069245, 0.7237414863069245

2. np.corrcoef(x,y)-----------------


array([[1.   , 0.724],
       [0.724, 1.   ]])

- df['english'] : 1.0
- df['mathematics'] : 0.9999999999999998
- 상관계수 : 0.7237414863069243, 0.7237414863069243

3. df.corr()-----------------


,english,mathematics
english,1.000,0.724
mathematics,0.724,1.000
